In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import torch
from typing import Any, Callable, Dict, List, Optional, Tuple, Union

# Original function

- 깃헙에 있는 함수를 그대로 가져온 것입니다.
- 이 스터디 덕분에 rotary position embedding에 대해서도 공부하게 되었는데, 그게 어떻게 구현되었는지 살펴보고자 했습니다.
- 원본의 input, output 뿐만 아니라 중간 data shape와 같은 흐름 또한 확인하고자했습니다.

In [3]:
def _rotate_half(x: torch.Tensor) -> torch.Tensor:
    """
    change sign so the last dimension becomes [-odd, +even]
    """
    x = x.view(x.shape[:-1] + torch.Size((2, x.shape[-1] // 2)))
    x1, x2 = x.unbind(dim=-2)
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb_pytorch(
    t: torch.Tensor,
    freqs: torch.Tensor,
    tensor_format: str = "sbhd",
    fused: bool = False,
    cu_seqlens: Union[torch.Tensor, None] = None,
) -> torch.Tensor:
    """
    Apply rotary positional embedding tensor to the input tensor.

    Parameters
    ----------
    t: torch.Tensor
        Input tensor of shape `[s, b, h, d]`, `[s, b, h, d]` or `[t, h, d]`, on which
        rotary positional embedding will be applied.
    freqs: torch.Tensor
        Rotary positional embedding tensor of shape `[s2, 1, 1, d2]` and dtype 'float',
        with `s2 >= s` and `d2 <= d`.
    fused: bool, default = False
        Whether to use a fused applying RoPE implementation.
    tensor_format: {'sbhd', 'bshd', 'thd'}, default = 'sbhd'
        is `bshd` if `t` is of shape `[bs, seq, ...]`, or `sbhd` if `t` is
        of shape `[seq, bs, ...]`. 'thd' is only supported when `fused` is True.
    cu_seqlens: torch.Tensor, default = None.
        Cumulative sum of sequence lengths in a batch for `t`, with shape [b + 1] and
        dtype torch.int32. Only valid when `tensor_format` is 'thd'.
    """
    if fused:
        assert (
            tensor_format != "thd" or cu_seqlens is not None
        ), "cu_seqlens must not be None when tensor_format is 'thd'."
        return FusedRoPEFunc.apply(t, freqs, tensor_format, cu_seqlens)

    assert tensor_format in ("sbhd", "bshd"), (
        "Only formats `sbhd` or `bshd` are supported for input tensor `t` "
        f"when fused is False, got {tensor_format}."
    )

    max_seq_len = freqs.shape[0]
    cur_seq_len = t.shape[1] if tensor_format == "bshd" else t.shape[0]

    # Only apply the rotary embeddings up to the sequence length of the running
    # input.
    assert cur_seq_len <= max_seq_len, (
        f"Rotary Embeddings only supported up to {max_seq_len} sequence length!"
    )
    freqs = freqs[:cur_seq_len]
    if tensor_format == "bshd":
        freqs = freqs.transpose(0, 1)  # [seq, 1, 1, dim] -> [1, seq, 1, dim]
    # cos/sin first then dtype conversion for better precision
    cos_ = torch.cos(freqs).to(t.dtype)
    sin_ = torch.sin(freqs).to(t.dtype)

    rot_dim = freqs.shape[-1]
    # ideally t_pass is empty so rotary pos embedding is applied to all tensor t
    t, t_pass = t[..., :rot_dim], t[..., rot_dim:]

    print('t shape(_rotate_half\'s input):', t.shape)
    # first part is cosine component
    # second part is sine component, need to change signs with _rotate_half method
    t = (t * cos_) + (_rotate_half(t) * sin_)
    return torch.cat((t, t_pass), dim=-1)

### Prepare input sample

In [4]:
s, b, h, d = 10, 2, 4, 16
t = torch.randn(s, b, h, d)

### hypothsis. s2 = s, d2 = d/2
s2, d2 = s, d//2
freqs = torch.randn(s2, 1, 1, d2)

tensor_format = "sbhd"

fused = False
cu_seqlens = None

# t, freqs, tensor_format, fused, cu_seqlens

t = t.to('cuda')
freqs = freqs.to('cuda')

### Result

In [6]:
original_result = apply_rotary_pos_emb_pytorch(
    t=t,
    freqs=freqs,
    tensor_format=tensor_format,
)

original_result.shape, original_result.mean(), original_result.std()

t shape(_rotate_half's input): torch.Size([10, 2, 4, 8])


(torch.Size([10, 2, 4, 16]),
 tensor(-0.0187, device='cuda:0'),
 tensor(0.9981, device='cuda:0'))

- 중간에 _rotate_half 함수가 있는데, apply_rotary_pos_emb_pytorch 함수를 마이그레이션 하기 위해서는 이 또한 변환해야 할 듯 합니다.
- 상대적으로 함수가 간단해보여 먼저 시도하였습니다.
- 주어진 input에 따른 _rotate_half 함수의 입력 shape로는 (10, 2, 4, 8) 형태가 들어가는 듯 합니다.

# Migrate '_rotate_half' function to triton

In [7]:
import triton
import triton.language as tl

### Define '_rotate_half_kernel'
- 아래 사진에서 3번째 term에 해당하는 (-x2 x2 -x4 x3 ...)을 구현한 것이 '_rotate_half_kernel'인 듯 합니다.
- triton은 생소하여 감을 잡는 데에 시간이 많이 걸린 듯 합니다... 
- 제가 구글링을 잘 못한 탓인지 자료가 많이 없어서 공부하기 어려움이 있었는데, 이 스터디를 통해 꼭 공부해보고 싶습니다.
- 총 몇 스레드에서 처리할 지, 한 스레드에서 얼마만큼의 블록을 담당할 지 정하는 부분이 있는데, block size의 경우 그냥 마지막 dimension의 크기로 놓는 경우가 많아 그렇게 했습니다만, 이 또한 좋은 기준이 있을까 궁금합니다
- 위에서는 (10, 2, 4, 8) shape이 입력으로 들어가는 것을 확인했으나, 제출용 노트북 파일에서는 그 값을 눈으로도 쉽게 볼 수 있게 하기 위하여 (2,1,2,4) shape을 입력으로 하였습니다. (10, 2, 4, 8)으로 해도 원본 파일과 같은 값을 출력합니다.

<img src="https://velog.velcdn.com/images/wkshin89/post/e00d46ed-3188-4e9b-bba1-afc908fde31a/image.png" width=600 height=400 />

In [8]:
@triton.jit
def _rotate_half_kernel(
    output_ptr, input_ptr,
    batch_size: tl.constexpr, seq_len: tl.constexpr, head_num: tl.constexpr, d_model: tl.constexpr,
    BLOCK_SIZE: tl.constexpr
):
    idx = tl.program_id(0) * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
    total_elements = batch_size * seq_len * head_num * d_model
    mask = idx < total_elements

    dim_idx = idx % d_model
    half_dim = d_model // 2
    is_second_half = dim_idx >= half_dim

    swapped_idx = tl.where(is_second_half, idx - half_dim, idx + half_dim)
    data = tl.load(input_ptr + idx, mask=mask)
    data_swapped = tl.where(is_second_half, -data, data)

    tl.store(output_ptr + swapped_idx, data_swapped, mask=mask)

### prepare sample input

In [9]:
batch_size = 2
seq_len = 1
head_num = 2
d_model = 4

input_tensor = torch.randn(batch_size, seq_len, head_num, d_model, device='cuda')
input_tensor

tensor([[[[-0.9488,  0.8871, -1.3509,  0.5680],
          [ 0.3531,  0.5859, -0.6028, -1.1586]]],


        [[[ 0.6844,  0.4549, -0.4418,  2.3851],
          [-0.9007, -0.2924,  0.6402,  1.2308]]]], device='cuda:0')

In [10]:
length = len(input_tensor)
stride = input_tensor.stride(0)
grid = (length,)
BLOCK_SIZE = stride
rotated_tensor_triton = torch.empty_like(input_tensor)
rotated_tensor_triton

tensor([[[[-0.9488,  0.8871, -1.3509,  0.5680],
          [ 0.3531,  0.5859, -0.6028, -1.1586]]],


        [[[ 0.6844,  0.4549, -0.4418,  2.3851],
          [-0.9007, -0.2924,  0.6402,  1.2308]]]], device='cuda:0')

In [11]:
total_elements = batch_size * seq_len * head_num * d_model

BLOCK_SIZE = d_model
# BLOCK_SIZE = 1
grid_size = (total_elements + BLOCK_SIZE - 1) // BLOCK_SIZE
# grid_size

### Compare outputs

In [12]:
rotated_tensor_pytorch = _rotate_half(input_tensor)

_rotate_half_kernel[(grid_size,)](
    rotated_tensor_triton, input_tensor, 
    batch_size=batch_size, seq_len=seq_len, head_num=head_num, d_model=d_model,
    BLOCK_SIZE=BLOCK_SIZE
)

torch.testing.assert_close(rotated_tensor_pytorch, rotated_tensor_triton)

In [13]:
input_tensor

tensor([[[[-0.9488,  0.8871, -1.3509,  0.5680],
          [ 0.3531,  0.5859, -0.6028, -1.1586]]],


        [[[ 0.6844,  0.4549, -0.4418,  2.3851],
          [-0.9007, -0.2924,  0.6402,  1.2308]]]], device='cuda:0')

In [14]:
rotated_tensor_pytorch

tensor([[[[ 1.3509, -0.5680, -0.9488,  0.8871],
          [ 0.6028,  1.1586,  0.3531,  0.5859]]],


        [[[ 0.4418, -2.3851,  0.6844,  0.4549],
          [-0.6402, -1.2308, -0.9007, -0.2924]]]], device='cuda:0')

In [15]:
rotated_tensor_triton

tensor([[[[ 1.3509, -0.5680, -0.9488,  0.8871],
          [ 0.6028,  1.1586,  0.3531,  0.5859]]],


        [[[ 0.4418, -2.3851,  0.6844,  0.4549],
          [-0.6402, -1.2308, -0.9007, -0.2924]]]], device='cuda:0')

### Compare processing time
- 몇 줄 되지도 않는 함수고, 그리 어려운 알고리즘도 아니었건만 짜는데 꽤 오래 걸렸는데...
- pytorch high level 함수의 조합보다 2배 가까이 느린건 좀 충격입니다.
- 일단 정상 동작한다는 점에 의의를 둬야 할 듯 합니다...

In [16]:
%timeit -n 100 -r 100 rotated_tensor_pytorch = _rotate_half(input_tensor)

The slowest run took 4.17 times longer than the fastest. This could mean that an intermediate result is being cached.
23.7 µs ± 8.04 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)


In [17]:
%timeit -n 100 -r 100 _rotate_half_kernel[(grid_size,)]( \
    rotated_tensor_triton, input_tensor, \
    batch_size=batch_size, seq_len=seq_len, head_num=head_num, d_model=d_model, \
    BLOCK_SIZE=BLOCK_SIZE \
)

47.4 µs ± 12 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
